In [84]:
import pandas as pd
import html5lib
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from tqdm import tqdm

In [85]:
def load_raw():
    dataframes=[]
    for k in tqdm(range(2001,2019)):
        for i in range(1,13):
            df = pd.read_html('./data/'+str(k)+'Y_'+str(i)+'M.xls',encoding='utf-8',header=0)
            dataframes.append(df[0])
    return dataframes

In [86]:
dataframes=load_raw()

100%|██████████| 18/18 [00:38<00:00,  4.35s/it]


In [87]:
df_nans=[]
for df in dataframes:
    df_nan=df[df.isna().any(1)]
    df_nan=df_nan.dropna(axis='columns')
    # 2007/12 2013/12 change
    df_nan.rename(columns={'類型代號':"基金名稱","基金統編":"基金規模","基金規模":"本月定時定額扣款筆數","單位淨值":"本月定時定額扣款人數","本月定時定額扣款筆數":"本月定時定額扣款金額"})
    df_nans.append(df_nan)

In [88]:
df_corrects=[]
df_filter = []
# for df in dataframes:
#     df_correct = df[~df.isna().any(1)]
#     df_corrects.append(df_correct)
for df in tqdm(dataframes):
    df.rename(columns={"基金規模(台幣)":"基金規模","單位淨值(台幣)":"單位淨值","本月定時定額扣款金額(台幣)":"本月定時定額扣款金額"}, inplace=True)
    df_correct = df[~df.isna().any(1)]
    df_filter = []
    for name in df_correct['基金名稱']:
        if ("中國" in name or "俄羅斯" in name or "巴西" in name or "印度" in name or "醫療" in name or "生技" in name) :
            if("中國信託" in name):
                continue
            filter1 = (df_correct['類型代號'] == "AA2")
            filter2 = (df_correct['類型代號'] == "2")
            filter3 = (df_correct['基金名稱'] == name)
            df_filter.append(df_correct[((filter1 | filter2) & filter3)])
    df_corrects.append(pd.concat(df_filter))
df_corrects[0]# 0=2001

100%|██████████| 216/216 [00:23<00:00,  1.90it/s]


,類型代號,基金統編,基金名稱,基金規模,單位淨值,本月定時定額扣款筆數,本月定時定額扣款人數,本月定時定額扣款金額
86,2,97990289,元大中國基金,436898521,6.54,8,8.0,63000.0


In [89]:
def plot_image(id):
    navs=[]
    money=[]
    num=[]
    people=[]
    scales=[]
    id = str(id)
    id = id.zfill(8)
    for i in range(0,2*12):
        try:
            navs.append(df_corrects[i][df_corrects[i]['基金統編']==id]['單位淨值'].values[0])
            money.append(df_corrects[i][df_corrects[i]['基金統編']==id]['本月定時定額扣款金額'].values[0])
            num.append(df_corrects[i][df_corrects[i]['基金統編']==id]['本月定時定額扣款筆數'].values[0])
            people.append(df_corrects[i][df_corrects[i]['基金統編']==id]['本月定時定額扣款筆數'].values[0])
            scales.append(df_corrects[i][df_corrects[i]['基金統編']==id]['基金規模'].values[0])
        except:
            pass
    plt.plot(navs)
    plt.title('Unit net value')
    plt.ylabel('Value')
    plt.xlabel('Months')
    plt.show()
    plt.title('SIP value')
    plt.plot(money)
    plt.ylabel('Value')
    plt.xlabel('Months')
    plt.show()
    plt.title('SIP Number of records')
    plt.plot(num)
    plt.ylabel('Value')
    plt.xlabel('Months')
    plt.show()
    plt.title('SIP Number of people')
    plt.plot(people)
    plt.ylabel('Value')
    plt.xlabel('Months')
    plt.show()
    plt.title('SIP Scale')
    plt.plot(scales)
    plt.ylabel('Value')
    plt.xlabel('Months')
    plt.show()
    

## Function of calculating rising or falling

In [90]:
def monthFilter(month, length, data):
    subData = np.array(data[(month - (month % 6)) : (month + (5-(month % 6)) + 1)])
    value=(data[month] - np.mean(subData)) / np.std(subData)
    if(value> 1 or value < -1):
        return value
    else:
        return 0

## This function will get various people,aliveList,variations

In [91]:
def preprocessing():
    id_list = []
    id_temp = []
    aliveList=[]
    uid=[]
    upeople=[]
    peopleChinaList = []
    peopleIndiaList = []
    peopleBrazilList = []
    peopleRussiaList = []
    peopleMedicalList = []
    markMonthChinaList =[]
    markMonthIndiaList = []
    markMonthBrazilList = []
    markMonthRussiaList = []
    markMonthlMedicalList = []
    MonthChinaValue=[]
    MonthIndiaValue=[]
    MonthRussiaValue=[]
    MonthBrazilValue=[]
    for df in df_corrects:
        for filter_id in df['基金統編']:
#             id = str(filter_id)
#             id = id.zfill(8)
            id_temp.append(filter_id)
    id_list = list(set(id_temp))
    for id in id_list:
            isAlive = True
            navs=[]
            money=[]
            num=[]
            people=[]
            scales=[]
            name = ''
            isSaveList = []
            for i in range((17-3)*12 + 4,17*12 + 4):
                try:
                    name = df_corrects[i][df_corrects[i]['基金統編']==id]['基金名稱'].values[0]
                    navs.append(df_corrects[i][df_corrects[i]['基金統編']==id]['單位淨值'].values[0])
                    money.append(df_corrects[i][df_corrects[i]['基金統編']==id]['本月定時定額扣款金額'].values[0])
                    num.append(df_corrects[i][df_corrects[i]['基金統編']==id]['本月定時定額扣款筆數'].values[0])
                    people.append(df_corrects[i][df_corrects[i]['基金統編']==id]['本月定時定額扣款人數'].values[0])
                    scales.append(df_corrects[i][df_corrects[i]['基金統編']==id]['基金規模'].values[0])
                    if(df_corrects[i][df_corrects[i]['基金統編']==id]['本月定時定額扣款人數'].values[0]) == 0 :
                        isAlive = False
                    if(i == 17*12 + 3 and isAlive):
                        aliveList.append(df_corrects[i][df_corrects[i]['基金統編']==id]['基金名稱'].values[0])
                        uid.append(df_corrects[i][df_corrects[i]['基金統編']==id]['基金統編'].values[0])
                
                except:
                    isAlive = False
                    navs.append(0)
                    money.append(0)
                    num.append(0)
                    people.append(0)
                    scales.append(0)
                    break
            funding={}
            funding
            if(isAlive):
                for i in range(0, len(people)):
                    if("中國" in name):
                        try:
                            peopleChinaList[i] += people[i]
                        except:
                            peopleChinaList.append(people[i])
                    elif("印度" in name):
                        try:
                            peopleIndiaList[i] += people[i]
                        except:
                            peopleIndiaList.append(people[i])
                    elif("巴西" in name):
                        try:
                            peopleBrazilList[i] += people[i]
                        except:
                            peopleBrazilList.append(people[i])
                    elif("俄羅斯" in name):
                        try:
                            peopleRussiaList[i] += people[i]
                        except:
                            peopleRussiaList.append(people[i])
                    else:
                        try:
                            peopleMedicalList[i] += people[i] 
                        except:
                            peopleMedicalList.append(people[i])
#                 plt.plot(navs)
#                 plt.title(id + 10*' ' +'Unit net value')
#                 plt.ylabel('Value')
#                 plt.xlabel('Months')
#                 plt.show()
#                 plt.title(id + 10*' ' +'SIP value')
#                 plt.plot(money)
#                 plt.ylabel('Value')
#                 plt.xlabel('Months')
#                 plt.show()
#                 plt.title(id + 10*' ' +'SIP Number of records')
#                 plt.plot(num)
#                 plt.ylabel('Value')
#                 plt.xlabel('Months')
#                 plt.show()
#                 plt.title(id + 10*' ' +'SIP Number of people')
#                 plt.plot(people)
#                 plt.ylabel('Value')
#                 plt.xlabel('Months')
#                 plt.show()
#                 plt.title(id + 10*' ' +'SIP Scale')
#                 plt.plot(scales)
#                 plt.ylabel('Value')
#                 plt.xlabel('Months')
#                 plt.show()

    # Calculate people and value
    
    china={}
    for i in range(0, len(peopleChinaList)):
        value=monthFilter(i, 6, peopleChinaList)
        if value!=0:
            markMonthChinaList.append(i)
            MonthChinaValue.append(value)
    china={
        'Month_Index':markMonthChinaList,
        'Value':MonthChinaValue
    }
    brazil={}
    for i in range(0, len(peopleBrazilList)):
        value=monthFilter(i, 6, peopleBrazilList)
        if value!=0:
            markMonthBrazilList.append(i)
            MonthBrazilValue.append(value)
    brazil={
        'Month_Index':markMonthBrazilList,
        'Value':MonthBrazilValue
    }
    for i in range(0, len(peopleIndiaList)):
        value=monthFilter(i, 6, peopleIndiaList)
        if value!=0:
            markMonthIndiaList.append(i)
            MonthIndiaValue.append(value)
    india={
        'Month_Index':markMonthBrazilList,
        'Value':MonthBrazilValue
    }
    for i in range(0, len(peopleRussiaList)):
        value=monthFilter(i, 6, peopleIndiaList)
        if value!=0:
            markMonthIndiaList.append(i)
            MonthIndiaValue.append(value)
    russia={
        'Month_Index':markMonthRussiaList,
        'Value':MonthRussiaValue
    }
    variation={
        'China':pd.DataFrame(china),
        'Brazil':pd.DataFrame(brazil),
        'India':pd.DataFrame(india),
        'Russia':pd.DataFrame(russia)
    }
    
#     for i in range(0, len(peopleMedicalList)):
#         if(monthFilter(i, 6, peopleMedicalList)):
#             markMonthlMedicalList.append(i)
#     plt.title('SIP Number of people in Medical')
#     plt.plot(peopleMedicalList)
#     plt.ylabel('Value')
#     plt.xlabel('Months')
#     plt.show()
    china_df=pd.DataFrame(china)
    dict={}
    start_date = "2016/04" 
    stop_date="2019/03"
    start=datetime.strptime(start_date,"%Y/%m")
    stop=datetime.strptime(stop_date,"%Y/%m")
    arr_smonth=[]
    while start<=stop:
        arr_smonth.append(start.strftime('%Y/%m')) # Return a string representing the date and time
        start = start +  relativedelta(months=1)
    percentage_c=[]
    percentage_b=[]
    percentage_r=[]
    percentage_i=[]
    for i in range(len(peopleChinaList)):
        total=peopleChinaList[i]+peopleIndiaList[i]+peopleRussiaList[i]+peopleBrazilList[i]
        percentage_c.append((peopleChinaList[i]/total)*100)
        percentage_b.append((peopleBrazilList[i]/total)*100)
        percentage_r.append((peopleRussiaList[i]/total)*100)
        percentage_i.append((peopleIndiaList[i]/total)*100)
        
        
    
    dict={
        'Date':arr_smonth,
        'China':peopleChinaList,
        'India':peopleIndiaList,
        'Russia':peopleRussiaList,
        'Brazil':peopleBrazilList,
        'China%':percentage_c,
        'India%':percentage_i,
        'Russia%':percentage_r,
        'Brazil%':percentage_b
    }
    alive={
        "name":aliveList,
        "uid":uid
    }
    
 
    peopleDF=pd.DataFrame(dict)
    aliveDF=pd.DataFrame(alive)
    return aliveList,aliveDF,variation,peopleDF



In [92]:
aliveList,aliveDF,varation ,people= preprocessing()
varation['China']

,Month_Index,Value
0,0,-1.856616
1,9,-2.054153
2,12,1.923390
3,17,-1.151120
4,20,-1.889533
5,22,1.122276
6,24,-1.470169
7,29,1.457625
8,33,-1.806295


In [93]:
people

,Date,China,India,Russia,Brazil,China%,India%,Russia%,Brazil%
0,2016/04,19451.0,2267.0,2104.0,3244.0,71.865071,8.375822,7.773590,11.985517
1,2016/05,22223.0,2247.0,2058.0,3210.0,74.729303,7.555989,6.920438,10.794270
2,2016/06,22482.0,2231.0,2040.0,3153.0,75.175550,7.460041,6.821374,10.543035
3,2016/07,22306.0,2442.0,1965.0,2971.0,75.144859,8.226654,6.619728,10.008759
4,2016/08,20813.0,2413.0,1881.0,2754.0,74.702990,8.660852,6.751373,9.884785
5,2016/09,21304.0,2360.0,1811.0,2597.0,75.890567,8.406954,6.451268,9.251211
6,2016/10,21493.0,2390.0,1738.0,2490.0,76.457614,8.502010,6.182633,8.857743
7,2016/11,21451.0,2314.0,1710.0,2375.0,77.023339,8.308797,6.140036,8.527828
8,2016/12,20909.0,2231.0,1663.0,2222.0,77.369103,8.255319,6.153562,8.222017
9,2017/01,17905.0,2046.0,1535.0,2062.0,76.036181,8.688636,6.518600,8.756582


In [94]:
newAliveList = []
for name in aliveList:
    if("新臺幣" in name):
        name = name.replace("新臺幣", "台幣")
    if("新台幣" in name):
        name = name.replace("新台幣", "台幣")
    if("台幣類型" in name):
        name = name.replace("台幣類型", "台幣")
    if("N" in name):
        name = name.replace("N", "N類型")
    if("-N類型" in name):
        name = name.replace("-N類型", "")
        name = name.replace("基金", "基金N類型")
    if("級別" in name):
        if("台幣" in name):
            name = name.replace("台幣級別", "(台幣)")
        if("人民幣" in name):
            name = name.replace("人民幣級別", "(人民幣)")
        if("美元" in name):
            name = name.replace("美元級別", "(美元)")
    if("-" in name):
        name = name.replace("-", "(", 1) + ")"
    if("人民幣類型" in name):
        name = name.replace("類型", "")
    if("兆豐國際中國A股基金(美金)" in name):
        name = name.replace("美金", "美元")
    if("安聯中國策略基金" in name):
        name = name + "(台幣)"
    if("台新中國精選中小基金" in name):
        name = name + "(台幣)"
    if("摩根中國A股基金" in name):
        if("摩根中國A股基金(美元)" in name):
            pass
        else:
            name = name + "(台幣)"
    if("瀚亞中國" in name):
        name = name.replace("中國", "中國A股")
    if("野村新興傘型基金之大俄羅斯基金" in name):
        name = name.replace("野村新興傘型基金之大俄羅斯基金", "野村大俄羅斯基金")
    if("野村雙印傘型基金之印度潛力基金" in name):
        name = name.replace("野村雙印傘型基金之印度潛力基金", "野村印度潛力基金")
    newAliveList.append(name)
# newAliveList

In [95]:
newAliveList

['安聯中國策略基金(台幣)',
 '群益全球關鍵生技基金(台幣)',
 '野村中國機會基金',
 '瀚亞巴西基金',
 '台新印度基金',
 '復華中國新經濟A股基金(台幣)',
 '安聯全球生技趨勢基金(台幣)',
 '保德信全球醫療生化基金(台幣)',
 '日盛中國內需動力基金',
 '群益中國新機會基金(台幣)',
 '安聯中國東協基金',
 '兆豐國際中國A股基金(台幣)',
 '德信中國精選成長基金',
 '台新中國精選中小基金(台幣)',
 '國泰中國內需增長基金(台幣)',
 '第一金中國世紀基金(台幣)',
 '野村印度潛力基金',
 '兆豐國際中國A股基金(美元)',
 '野村全球生技醫療基金',
 '摩根中國亮點基金',
 '元大印度基金',
 '野村大俄羅斯基金',
 '野村巴西基金']

In [96]:
# df_corrects[172][df_corrects[172]['基金名稱'] =='瀚亞巴西基金']['本月定時定額扣款人數']

In [97]:
aliveDF

,name,uid
0,安聯中國策略基金,25589337
1,群益全球關鍵生技基金-新臺幣,38486708A
2,野村中國機會基金,25530596
3,瀚亞巴西基金,25633559
4,台新印度基金,26322646
5,復華中國新經濟A股基金-新臺幣,42303511A
6,安聯全球生技趨勢基金-新臺幣,14693593A
7,保德信全球醫療生化基金-新臺幣,14692638A
8,日盛中國內需動力基金,26323737
9,群益中國新機會基金-新臺幣,38524463A


In [98]:
def country_classifier(newAliveList):
    country=dict()
    china_id=[]
    china_n=[]
    india_id=[]
    india_n=[]
    brazil_id=[]
    brazil_n=[]
    russia_id=[]
    russia_n=[]
    for funding,uid in zip(newAliveList['name'],newAliveList['uid']):
        if '中國' in funding:
            china_id.append(uid)
            china_n.append(funding)
        elif '巴西' in funding:
            brazil_id.append(uid)
            brazil_n.append(funding)
        elif '印度' in funding:
            india_id.append(uid)
            india_n.append(funding)
        elif '俄羅斯' in funding:
            russia_id.append(uid)
            russia_n.append(funding)
    country_id={
        'China':china_id,
        'Brazil':brazil_id,
        'India':india_id,
        'Russia':russia_id
    }
    country_n={
        'China':china_n,
        'Brazil':brazil_n,
        'India':india_n,
        'Russia':russia_n
        
    }
    return country_n,country_id

In [99]:
result_n,result_id=country_classifier(aliveDF)

In [100]:
def get_funding(dataframe):
    start_date = "2016/04" 
    stop_date="2019/03"
    start=datetime.strptime(start_date,"%Y/%m")
    stop=datetime.strptime(stop_date,"%Y/%m")
    arr_smonth=[]
    while start<=stop:
        arr_smonth.append(start.strftime('%Y/%m')) 
        start = start +  relativedelta(months=1)
    sip_people={}
    sip_people['Date']=arr_smonth
    for name,item in zip(dataframe['name'],dataframe['uid']):
        arr=[]
        for i in range(14*12+4,17*12+4):
            a=df_corrects[i][df_corrects[i]['基金統編']==item]['本月定時定額扣款人數'].values[0]
            arr.append(a)
        sip_people[item]=arr        
    return pd.DataFrame(sip_people)

In [101]:
result_n['China']

['安聯中國策略基金',
 '野村中國機會基金',
 '復華中國新經濟A股基金-新臺幣',
 '日盛中國內需動力基金',
 '群益中國新機會基金-新臺幣',
 '安聯中國東協基金',
 '兆豐國際中國A股基金(台幣)',
 '德信中國精選成長基金',
 '台新中國精選中小基金',
 '國泰中國內需增長基金台幣級別',
 '第一金中國世紀基金-新臺幣',
 '兆豐國際中國A股基金(美金)',
 '摩根中國亮點基金']

In [102]:
result_id['China']

['25589337',
 '25530596',
 '42303511A',
 '26323737',
 '38524463A',
 '25680150',
 '38592074A',
 '25689835',
 '25691213',
 '26317735A',
 '25620512A',
 '38592074B',
 '25627402']

In [103]:
def get_moneys(ids):
    sip_moneys={}
    for item in ids:
        sip_money=[]
        for i in range(14*12+4,17*12+4):
                    a=df_corrects[i][df_corrects[i]['基金統編']==item]['本月定時定額扣款金額'].values[0]
                    sip_money.append(a)
                    sip_moneys[item]=sip_money
    return pd.DataFrame(sip_moneys)

In [104]:
def get_people(ids):
    sip_people={}
    for item in ids:
        sip_person=[]
        for i in range(14*12+4,17*12+4):
                    a=df_corrects[i][df_corrects[i]['基金統編']==item]['本月定時定額扣款人數'].values[0]
                    sip_person.append(a)
                    sip_people[item]=sip_person
    return pd.DataFrame(sip_people)

In [106]:
china_sip_people=get_people(result_id['China'])
for item in  result_id['China']:
    china_sip_people[item]= china_sip_people[item]/china_sip_people[item].max()
china_sip_people.columns = result_n['China']
china_sip_people

,安聯中國策略基金,野村中國機會基金,復華中國新經濟A股基金-新臺幣,日盛中國內需動力基金,群益中國新機會基金-新臺幣,安聯中國東協基金,兆豐國際中國A股基金(台幣),德信中國精選成長基金,台新中國精選中小基金,國泰中國內需增長基金台幣級別,第一金中國世紀基金-新臺幣,兆豐國際中國A股基金(美金),摩根中國亮點基金
0,0.625698,0.850359,0.002220,0.953043,0.766253,1.000000,0.696557,0.93750,0.941176,0.629124,0.771238,0.75,0.963737
1,0.691620,0.860298,0.079726,0.986087,0.942967,0.998456,0.715925,0.96875,0.978610,0.670046,0.996940,0.70,1.000000
2,0.728492,0.829928,0.111728,1.000000,1.000000,0.971429,0.728838,0.84375,0.957219,0.661751,1.000000,0.60,0.990874
3,0.769832,0.831033,0.118942,0.987826,0.981678,0.938224,0.721664,0.81250,0.893048,0.656959,0.994060,0.60,0.976465
4,0.658101,0.792932,0.118942,0.966957,0.812352,0.744402,0.713773,0.75000,0.909091,0.623410,0.967783,0.55,0.940682
5,0.806704,0.776367,0.122457,0.947826,0.916371,0.952124,0.710904,0.68750,0.903743,0.622120,0.935925,0.55,0.928674
6,0.795531,0.765323,0.130226,0.932174,0.957742,0.961390,0.717360,0.75000,0.941176,0.623226,0.941865,0.55,0.923151
7,0.777654,0.753727,0.134295,0.926957,0.987884,0.938996,0.716643,0.75000,1.000000,0.619908,0.933765,0.55,0.912584
8,0.769832,0.753727,0.140585,0.913043,0.981383,0.920463,0.701578,0.75000,0.978610,0.602581,0.879410,0.45,0.890490
9,0.720670,0.712314,0.142434,0.859130,0.804078,0.897297,0.696557,0.84375,0.962567,0.583594,0.744960,0.50,0.559558


In [108]:
brazil_sip_people=get_people(result_id['Brazil'])
for item in  result_id['Brazil']:
    brazil_sip_people[item]= brazil_sip_people[item]/brazil_sip_people[item].max()
brazil_sip_people.columns = result_n['Brazil']
brazil_sip_people

,瀚亞巴西基金,野村巴西基金
0,1.000000,1.000000
1,0.983312,0.995255
2,0.961489,0.981613
3,0.901155,0.929419
4,0.830552,0.865955
5,0.788190,0.811981
6,0.747754,0.785884
7,0.710526,0.752076
8,0.654685,0.712930
9,0.609756,0.659549


In [109]:
russia_sip_people=get_people(result_id['Russia'])
for item in  result_id['Russia']:
    russia_sip_people[item]= russia_sip_people[item]/russia_sip_people[item].max()
russia_sip_people.columns = result_n['Russia']
russia_sip_people

,野村新興傘型基金之大俄羅斯基金
0,1.000000
1,0.978137
2,0.969582
3,0.933935
4,0.894011
5,0.860741
6,0.826046
7,0.812738
8,0.790399
9,0.729563


In [110]:
india_sip_people=get_people(result_id['India'])
for item in  result_id['India']:
    india_sip_people[item]= india_sip_people[item]/india_sip_people[item].max()
india_sip_people.columns = result_n['India']
india_sip_people

,台新印度基金,野村雙印傘型基金之印度潛力基金,元大印度基金
0,0.916955,0.993939,0.770758
1,0.916955,0.984416,0.762635
2,0.951557,0.971429,0.752708
3,0.968858,1.000000,0.908845
4,0.989619,0.979221,0.898917
5,0.986159,0.954113,0.878159
6,1.000000,0.974026,0.880866
7,0.955017,0.934199,0.865523
8,0.927336,0.895238,0.838448
9,0.927336,0.877056,0.690433


In [111]:
f = open('russia_people.json','w')
f.write(russia_sip_people.to_json(force_ascii=False))
f.close()

In [112]:
f = open('india_people.json','w')
f.write(india_sip_people.to_json(force_ascii=False))
f.close()

In [113]:
f = open('china_people.json','w')
f.write(china_sip_people.to_json(force_ascii=False))
f.close()

In [114]:
f = open('brazil_people.json','w')
f.write(brazil_sip_people.to_json(force_ascii=False))
f.close()

In [76]:
china_sip_money=get_moneys(result_id['China'])
for item in  result_id['China']:
    china_sip_money[item]= china_sip_money[item]/china_sip_money[item].max()
china_sip_money.columns = result_n['China']

In [77]:
brazil_sip_money=get_moneys(result_id['Brazil'])
for item in  result_id['Brazil']:
    brazil_sip_money[item]= brazil_sip_money[item]/brazil_sip_money[item].max()
brazil_sip_money.columns = result_n['Brazil']

In [78]:
russia_sip_money=get_moneys(result_id['Russia'])
for item in  result_id['Russia']:
    russia_sip_money[item]= russia_sip_money[item]/russia_sip_money[item].max()
russia_sip_money.columns = result_n['Russia']
# russia_sip_money

In [79]:
india_sip_money=get_moneys(result_id['India'])
for item in  result_id['India']:
    india_sip_money[item]= india_sip_money[item]/india_sip_money[item].max()
india_sip_money.columns = result_n['India']

In [36]:
df=get_funding(aliveDF)
df_china=df[result_id['China']]
df_china.columns = [result_n['China']]
df_china
print(df_china.to_json(force_ascii=False))
f = open('china.json','w')
f.write(df_china.to_json(force_ascii=False))
f.close()

{"["日盛中國內需動力基金"]":{"0":548,"1":567,"2":575,"3":568,"4":556,"5":545,"6":536,"7":533,"8":525,"9":494,"10":515,"11":511,"12":498,"13":488,"14":498,"15":493,"16":484,"17":487,"18":463,"19":437,"20":425,"21":415,"22":426,"23":420,"24":398,"25":404,"26":394,"27":396,"28":398,"29":408,"30":410,"31":410,"32":409,"33":392,"34":406,"35":395},"["野村中國機會基金"]":{"0":1540,"1":1558,"2":1503,"3":1505,"4":1436,"5":1406,"6":1386,"7":1365,"8":1365,"9":1290,"10":1285,"11":1259,"12":1243,"13":1231,"14":1233,"15":1244,"16":1235,"17":1233,"18":1219,"19":1220,"20":1216,"21":1215,"22":1223,"23":1199,"24":1201,"25":1222,"26":1259,"27":1288,"28":1343,"29":1423,"30":1557,"31":1657,"32":1685,"33":1746,"34":1794,"35":1811},"["第一金中國世紀基金-新臺幣"]":{"0":4285,"1":5539,"2":5556,"3":5523,"4":5377,"5":5200,"6":5233,"7":5188,"8":4886,"9":4139,"10":5181,"11":4698,"12":4622,"13":4598,"14":4557,"15":4614,"16":4635,"17":4633,"18":4521,"19":4423,"20":3845,"21":3896,"22":5240,"23":4301,"24":4269,"25":4285,"26":4155,"27":3982,"28":393

In [37]:
df_india=df[result_id['India']]
df_india.columns = [result_n['India']]
df_india
f = open('india.json','w')
f.write(df_india.to_json(force_ascii=False))
f.close()

In [39]:
df_russia=df[result_id['Russia']]
df_russia.columns = [result_n['Russia']]
df_russia
f = open('russia.json','w')
f.write(df_russia.to_json(force_ascii=False))
f.close()

In [38]:
df_brazil=df[result_id['Brazil']]
df_brazil.columns = [result_n['Brazil']]
f = open('brazil.json','w')
f.write(df_brazil.to_json(force_ascii=False))
f.close()

In [80]:
f = open('russia_percentage.json','w')
f.write(russia_sip_money.to_json(force_ascii=False))
f.close()

In [81]:
f = open('india_percentage.json','w')
f.write(india_sip_money.to_json(force_ascii=False))
f.close()

In [82]:
f = open('brazil_percentage.json','w')
f.write(brazil_sip_money.to_json(force_ascii=False))
f.close()

In [83]:
f = open('china_percentage.json','w')
f.write(china_sip_money.to_json(force_ascii=False))
f.close()

In [ ]:
pd.set_option('display.max_rows', None)
df2 = pd.read_html("https://www.moneydj.com/funddj/yb/YP302000.djhtm?a=ET003001")
df3 = pd.read_html("https://www.moneydj.com/funddj/yb/yp302000.djhtm?a=ET003002")
df4 = pd.read_html("https://www.moneydj.com/funddj/yb/yp302000.djhtm?a=ET003003")
dfFinal = pd.concat([df2[4], df3[4], df4[4]], axis=0)
dfFinal

In [ ]:
for name in newAliveList:
    filterName = (dfFinal["基金名稱"]["基金名稱"] == name)
    try:
        print(dfFinal[filterName]["報酬率(%)"]["六個月"].values[0])
    except:
        print(name)